# Рекуррентные нейронные сети (RNN)

##  Особенности

До этого мы работали с методами машинного обучения, которые работают с признаковым описанием фиксированной длины.

Это верно про табличные данные &mdash; обычно каждый объект в таких данных описан фиксированным набором признаков, и новые признаки не могут появиться "вдруг". Мы точно знаем, что на вход нам приходит объект размера 100, а на выходе мы должны для него предсказать 1 число. 

Верно это и про данные с фотографий &mdash; обычно наша нейросеть учится на изображениях определенного разрешения. Да, иногда мы делаем нейросеть, которая способна работать с изображением почти любого разрешения, но добиваемся мы этого за счет вставки слоев global pooling, которые просто приводят любое признаковое описание, полученнное нашей нейросетью, а точнее, ее сверточной частью, ранее, к фиксированному размеру. 

Однако многие данные такой структурой не обладают. К примеру &mdash; тексты. Возьмем все абзацы из "Войны и Мира". Какие-то будут больше, какие-то меньше. И обрезать их как-то нельзя. Аналогично будет и для текстов из твиттера и тд. И что делать, если мы хотим предсказывать, например, эмоциональную окрашенность текста? 

Более того, а что делать, если, к примеру, на основе абзаца текста нам необходимо сгенерировать его краткое содержание? То есть и выход у нас может быть разной длины. Аналогично &mdash; мы хотим по данным о курсе валюты за прошлый год спрогнозировать курс валюты на следующий месяц по дням. 

<img src ="https://edunet.kea.su/repo/src/L01_Intro/img/mp/types3.png" width="700">

Таким образом, у нас есть целый набор задач, для которых необходима особая структура нейросети, позволяющая так или иначе принимать данные разного размера. 


## Типы задач

**Анализ временных рядов**
- Табличные данные
- Аннотирование изображений и видео (Image/Video captioning)
- Машинный перевод
- Распознавание текста
- Распознавание речи

**Генеративные модели**
- Генерация текста/речи (чат - боты)
- Генерация изображений

**Классификация**
- Изображения
- Блоки текста (Sentiment analysis)


Общее для задач - мы имеем возможность сохранять информацию, сформированную при обработке одного слова, и использовать ее, когда мы анализируем дальнейшие слова.

До прихода нейронных сетей предложения пытались описать при помощи набора правил - грамматик, которые довольно успешно могли генерировать новые осмысленные предложения. Так выглядит разбор при помощи грамматик простого предложения 


<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/1200px-ParseTree.png" width="800">

Однако грамматики плохо учитывали более глобальные связи. Например, очень тяжело было добиться того, чтобы в абзаце текста:

"Леша пришел домой. Он будет есть рыбу"

компьютер понял, что во втором предложении "Он" сооттветствует "Леше". 


Основная идея, которая заключается в RNN состоит в следующем &mdash; а давайте возьмем и всю последовательность пропустим через одну и туже нейросеть. 
Но при этом сама нейросеть кроме следующего элемента последовательности (например, слова в тексте), будет принимать еще один параметр &mdash; некий $h$, который в начале будет, например, вектором из нулей, а далее &mdash; значением, которое выдает сама нейросеть после обработки очередного элемента последовательности (**токена**)

В этом $h$ будет храниться hidden state, скрытое состояние, учитывающее и локальный, и глобальный контекст.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/1_SKGAqkVVzT6co-sZ29ze-g.png" width="800">


[ссылка](https://en.wikipedia.org/wiki/Recurrent_neural_network)

При этом наша нейросеть может выдавать некий ответ на каждом шаге, но мы можем:

 1. использовать только выданное на последнем (если нам нужно предсказать одно значение) &mdash; many-to-one

 2. мы можем подавать в наше нейросетку токены (когда кончился исходный сигнал &mdash; подаем нулевые токены), пока она не сгенеирует сигнал стоп (many-to-many, one-to-many)

 3. можем делать различные комбинации, игнорируя выходы нейросети в начале

В зависимости от структуры входов и выходов, можно выделить несколько типов рекуррентных нейронных сетей:

One-to-one &mdash; обычная нейронная сеть, RNN здесь не нужно

Более сложной является реализация «one to many», когда у нас есть всего один вход и нам необходимо сформировать несколько выходов. Такой тип нейронной сети актуален, когда мы говорим о генерации музыки или текстов Мы задаем начальное слово или начальный звук, а дальше модель начинает самостоятельно генерировать выходы, в качестве входа к очередной ячейке рассматривая выход с прошлой ячейки нейронной сети.

 Если мы рассматриваем задачу классификации, то актуальна схема «many to one». Мы должны проанализировать все входы нейронной сети и только в конце определиться с классом.
 
  Схему «many to many», когда количество выходов равно количеству входов нейронной сети, мы рассмотрели на примере с определением части речи. Такой вид используется также в задачах NER, которые мы обсудим в следующем видео. 
  
Ну и последней разновидностью нейронных сетей является сеть вида «many to many», когда количество выходов нейронной сети не равно количеству входов. Это актуально, к примеру, в машинном переводе, когда одна и та же фраза может иметь разное количество слов в разных языках (т.е. это реализует схему энкодер-декодер). Энкодер получает данные различной длины &mdash; например, предложение на английском языке. С помощью скрытых состояний он формирует из исходных данных вектор, который затем передаётся в декодер. Последний, в свою очередь, генерирует из полученного вектора выходные данные &mdash; исходную фразу, переведённую на другой язык.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-011-1.png" width="1000">

Можно объединять разные подходы. Сначала генерируем некий $h$, который содержит сжатую информацию о том, что было подано в нейросеть, а затем подаем его в нейросеть one-to-many, которая генерирует, к примеру, перевод того, текста, что был подан первой части нейросети. 

<img src ="https://camo.githubusercontent.com/784aa789a413acf72ff98ab1ae6c2e984a871a35/687474703a2f2f6564756e65742e6b65612e73752f7265706f2f7372632f4c30385f524e4e2f696d672f3439385f4641323031395f6c65637475726531322d3033302e706e67" width="1000">



## Базовый RNN блок

Попробуем подробнее разобраться, что же происходит в загадочном зелёном прямоугольнике с надписью RNN. Внутри него мы вычисляем рекуррентное соотношение с помощью функции $f$, которая зависит от весов $w$. Чтобы найти новое состояние $h_t$, мы берём предыдущее скрытое состояние $ h_{t-1} $, а также текущий ввод xt. Когда мы отправляем в модель следующие входные данные, полученное нами скрытое состояние $ h_t $ передаётся в эту же функцию, и весь процесс повторяется.

Чтобы генерировать вывод в каждый момент времени, в модель добавляются полносвязные слои, которые постоянно обрабатывают состояния $ h_t $ и выдают основанные на них прогнозы. При этом функция $f$ и веса $w$ остаются неизменными.

Самая простая реализация рекуррентной сети будет выглядеть следующим образом (Тангенс здесь используется для введения нелинейности в систему):

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-017.png" width="700">

Важное отличие от слоев с которыми мы уже сталкивались, сотоит в том что на выходе мы получаем два объекта Y и H.

**Y** - предсказание в текущий момент времени, например метка класса.

**H** - контекст в котором предсказание было сделанно. Он может использоваться для дальнейших предсказаний.

#### RNNCell

В Pytorch для вычисления $h_t$ используется модуль [RNNCell](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html)  

$y_t$ в нем не вычисляется, предполагается что для его получения в модель должен быть добавлен дополнительный линейный слой.

**`input_size`** &mdash;  рамер элемента последовательности.

В отличие от сверточных, это всегда вектор а не тензор, поэтому `input_size` &mdash;  скаляр.

**`hidden_size`** &mdash; тоже скаляр. Он задает размер скрытого состояния которое тоже является вектором. Фактически это количество нейронов в слое.


In [ ]:
import torch

rnn_cell = torch.nn.RNNCell(input_size = 3, hidden_size = 2)
dummy_sequence = torch.randn((1, 3)) # batch, input_size
h = rnn_cell(dummy_sequence) 
print(f"Out = h\n{h.shape} \n{h}")  # hidden state 

Внутри происходит примерно следующее:
Для понятности в данном примере опущена батчевая обработка.

In [ ]:
import numpy as np
from torch import nn

# Simple RNNcell without a bias and batch support
class SimplifiedRNNCell(nn.Module): 
  def __init__(self, input_size, hidden_size):
    super().__init__()
    # Init weight matrix, for simplicity omit bias
    self.W_hx = np.random.randn(input_size, hidden_size) * 0.0001 # hidden_size == number of neurons
    self.W_hh = np.random.randn(hidden_size, hidden_size) * 0.0001 # naive initialization
    self.h0 = np.zeros((hidden_size)) # Initial hidden state
  
  def forward(self,x,h = None): # Without a batch dimension
    if h is None:
      h = self.h0
    h = np.tanh(self.W_hx.T.dot(x)+self.W_hh.T.dot(h))
    return h 
  
simple_rnn_cell = SimplifiedRNNCell(input_size = 3, hidden_size = 2)
h = simple_rnn_cell(dummy_sequence[0]) # No batch 
print(f"Out = h\n{h.shape} \n{h}")


<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-023.png" width="900">

Однако в последовательности всегда несколько элементов. И надо применить алгоритм к каждому.


 Поэтому RNNCell напрямую не используется. Для него есть обертка: [RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html) который обеспечивает последовательный вызов RNNCell для всех элементов последовательности.

### RNN блок в Pytorch 

**Warning: batch dim is second!**

In [ ]:
from torch import nn
rnn = torch.nn.RNN(input_size = 3, hidden_size = 2) # batch_first = True
dummy_batched_seq = torch.randn((2, 1, 3)) # seq_len, batch , input_size
out, h = rnn(dummy_batched_seq) 
print("Out = \n", out.shape,"\n", out) # hidden state for each element of sequence
print("h = \n", h.shape, "\n", h) # hidden state for last element of sequence

Внутри происходит примерно следующее

In [ ]:
# Simple RNN without batching
import numpy as np
from torch import nn

class SimplifiedRNNLayer(nn.Module): 
  def __init__(self, input_size, hidden_size):
    super().__init__()
    self.rnn_cell = SimplifiedRNNCell(input_size, hidden_size)

  # Without a batch dimension x have sahape seq_len * input_size
  def forward(self,x, h = None):
    all_h = []
    for i in range(x.shape[0]):
      h = self.rnn_cell(x[i],h)
      all_h.append(h) 
    return  np.stack(all_h), h

simple_rnn = SimplifiedRNNLayer(input_size = 4, hidden_size = 2)
sequence = np.array([[0,1,2,0], [3,4,5,0]]) # batch with one sequence of two elements 

out, h = simple_rnn(sequence)
print("Out \n",out.shape,out) 
print("h \n", h.shape, h)

Давайте разберемся.

К данным добаляется еще одно измерение размер последовательности. Таким образом batch из 6 последовательностей по 5 элементов в каждой будет выглядеть так:

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/rnn_batch.png" width="600">

P.S. Размер самого элемента == 3

Внутри RNN модуля элеменым последовательности обрабатываются последовательно:

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gan/rnn_unrolled.jpeg" width="1000">

[ссылка]()

Веса при этом используются одни и те же.

In [ ]:
import torch

dummy_input = torch.randn((2,1,3)) #  seq_len, batch, input_size

print("RNNCell")
rnn_cell = torch.nn.RNNCell(3,2)
for t, p in rnn_cell.named_parameters():
  print(t, p.shape) 

cell_out = rnn_cell(dummy_input[0,:,:]) # take first element from sequence 
print("Out = h", cell_out) # one hidden state

print("RNN")
rnn = torch.nn.RNN(3,2)
for t, p in rnn_cell.named_parameters():
  print(t, p.shape) 

out, h = rnn(dummy_input)

print("Out", out) # h for all sequence element 
print("h", h) # h for last element 

### $\color{brown}{\text{*Stacked RNNs}}$

[Ссылка на статью What is num_layers in RNN module?](https://discuss.pytorch.org/t/what-is-num-layers-in-rnn-module/9843/2)

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/layers.png" width="700">

h - справа, out - сверху

In [ ]:
import torch

dummy_input = torch.randn((2,1,3)) #  seq_len,  batch, input_size
rnn = torch.nn.RNN(3,2,num_layers=3)

# Weights matrix sizes not changed!
for t, p in rnn_cell.named_parameters():
  print(t, p.shape) 

out, h = rnn(dummy_input)

print("Out", out.shape, out) # Hidden states for all elements from top layer
print("h", h.shape, h) # Hidden states for last element for all layers

### $\color{brown}{\text{*Bidirectional}}$
Последовательность можно пропустить через сетьдва раза в прямом и обратном направлении.

[Cсылка на статью Understanding RNN implementation in PyTorch](https://medium.com/analytics-vidhya/understanding-rnn-implementation-in-pytorch-eefdfdb4afdb)

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/bidirectional.png" width="850">

$s'_0$ - инициаизируется

In [ ]:
import torch

dummy_input = torch.randn((2, 1, 3)) #   seq_len, batch, input_size
rnn = torch.nn.RNN(3, 2, bidirectional=True, num_layers=1)

for t, p in rnn_cell.named_parameters():
  print(t, p.shape) 

out, h = rnn(dummy_input)

print("Out", out) # Concatenated Hidden states from both layers
print("h", h) # Hidden states last element from  both : 2*num_layers*hidden_state

### $\color{brown}{\text{*Добавление выходных весов (y_t)}}$
Давайте добавим выходные веса. Для этого придется программировать.Воспользуемся параметром batch_first = True что бы batch измерение оказалось на привычном нам месте.

In [ ]:
import torch
# Let's add output weights

class RNN_for_many_to_one(torch.nn.Module):
    def __init__(self, input_size, hidden, output_size):
        super().__init__()
        self.rnn = torch.nn.RNN(input_size, hidden, batch_first = True)
        self.fc1 = torch.nn.Linear(hidden, output_size)
    
    def forward(self, x):
        x, hidden = self.rnn(x)
        print(x.shape) # h for each element
        print(hidden.shape) 
        # we need only last output
        return self.fc1(hidden) # get last output equivalent to self.fc1(x[:,-1,:])
    
model2 = RNN_for_many_to_one(28, 128, 10) # input_size, hidden_dim, classes
dummy_input = torch.randn((8, 28, 28)) #  batch, seq_len, element_size
res = model2(dummy_input)
# 30 - batch, 50 - output_size
print(res.shape)

По умолчанию `batch_first=False` и batch измерение становится вторым!

## Пример обработки временного ряда

---
- [A Gentle Introduction to Handling a Non-Stationary Time Series in Python](https://www.analyticsvidhya.com/blog/2018/09/non-stationary-time-series-python/https://www.analyticsvidhya.com/blog/2018/09/non-stationary-time-series-python/)
- [A comprehensive beginner’s guide to create a Time Series Forecast](https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/)
- [Detecting stationarity in time series data](https://towardsdatascience.com/detecting-stationarity-in-time-series-data-d29e0a21e638)
- [Most useful Python functions for Time Series Analysis](https://towardsdatascience.com/most-useful-python-functions-for-time-series-analysis-ed1a9cb3aa8b)
- [Data transformations and forecasting models: what to use and whenData transformations and forecasting models: what to use and when](https://people.duke.edu/~rnau/whatuse.htmhttps://people.duke.edu/~rnau/whatuse.htm)

Что общего у прогнозирования потребления электроэнергии домохозяйствами, оценки трафика на дорогах в определенные периоды, прогнозировании паводков и прогнозировании цены, по которой акции будут торговаться на фондовой бирже?

Все они подпадают под понятие данных временных рядов! Вы не можете точно предсказать любой из этих результатов без компонента «время». И по мере того, как в мире вокруг нас генерируется все больше и больше данных, прогнозирование временных рядов становится все более важной областью применения методов ML и DL.



## Сделаем предикт с помощью Deep Learning
----

[Time Series Prediction with LSTM Using PyTorchTime Series Prediction with LSTM Using PyTorch](https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/content/courses/deeplearning/notebooks/pytorch/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb#scrollTo=NabsV8O5BBd5https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/content/courses/deeplearning/notebooks/pytorch/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb#scrollTo=NabsV8O5BBd5)

- Загрузка данных
- Шкалирование
- Формирование ансамблей данных 
- Traning
- Testing

### Загрузка данных

Air Passengers per month. 
[Dataset Air Passengers Number of air passengers per month](https://www.kaggle.com/rakannimer/air-passengers)


[Time Series Prediction with LSTM Using PyTorchTime Series Prediction with LSTM Using PyTorch](https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/content/courses/deeplearning/notebooks/pytorch/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb#scrollTo=NabsV8O5BBd5https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/content/courses/deeplearning/notebooks/pytorch/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb#scrollTo=NabsV8O5BBd5)


In [ ]:
# Dataloading
import pandas as pd

!if test -f ./airline-passengers.csv; then echo "Already downloaded"; else wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv; fi

dataset = pd.read_csv('airline-passengers.csv')
dataset.head()

In [ ]:
import matplotlib.pyplot as plt
training_data = dataset.iloc[:,1:2].values # перевели dataframe в numpy.array
# plotting
plt.figure(figsize=(12, 4))
plt.plot(training_data, label = 'Airline Passangers Data')
plt.grid()
plt.show()

### Шкалирование данных

In [ ]:
# Min - Max normalization
td_min = training_data.min()
td_max = training_data.max()
training_data -= td_min
training_data = training_data / td_max
print(training_data[:5])

### Формирование ансамблей данных

Поскольку мы хотим научиться предсказывать следующие значение на основе предидущих нам нужно подготовить данные соответствующим образом.

Разобьем весь массив данных на фрагменты вида

x -> y

где х - это подпоследовательность, например строки с 1-й по 8-ю, а y - это значение из 9-й строки, то самое которое мы хотим предсказать




In [ ]:
import numpy as np
import torch
import torch.nn as nn
# функция создания "ансамблей" данных
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)] # seq_len * elements
        _y = data[i+seq_length] # one element
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)
    
# установка длины ансамбля. от нее практически всегда зависит точность предикта и быстродействие
seq_length = 8 # сравните 2 и 32
x, y = sliding_windows(training_data, seq_length)
x[0], y[0]

Благодаря такому подходу мы можем работать с RNN моделью так же как работали со сверточными моделями. Подавая на вход такую подпоследовательность + результат.

### Разобьем на train и test

In [ ]:
train_size = int(len(y) * 0.8)
test_size = len(y) - train_size

dataX = torch.Tensor(np.array(x))
dataY = torch.Tensor(np.array(y))

trainX = torch.Tensor(np.array(x[0:train_size]))
trainY = torch.Tensor(np.array(y[0:train_size]))

testX = torch.Tensor(np.array(x[train_size:len(x)]))
testY = torch.Tensor(np.array(y[train_size:len(y)]))

print(trainX.shape, trainY.shape, testX.shape, testY.shape)

### Model


In [ ]:
class AirTrafficPredictor(nn.Module):

    def __init__(self, input_size, hidden_size):
        # hidden_size == number of neurons 
        super().__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first = True)
        self.fc = nn.Linear(hidden_size, 1) # Predict only one value

    def forward(self, x):
        #print("x: ",x.shape) # 108 x 8 x 1 : [batch_size, seq_len, input_size] 
        out, h = self.rnn(x) 
        #print("out: ", out.shape) # 108 x 8 x 4 : [batch_size, seq_len, hidden_size] Useless!
        #print("h : ", h.shape) # 1 x 108 x 4 [ num_layers, batch_size, hidden_size]
        y = self.fc(h)
        #print("y",y.shape) # 1 x 108 x 1
        return y, h

### Обучение

Благодаря подготовке данных процесс обучения не будет отличаться от того что мы использовали на прошедших занятиях.

В силу того что датасет маленький, все данные поместились в один batch, итерирования по batch-ам в явном виде здесь не происходит.

In [ ]:
def time_series_train(model):
  num_epochs = 2000
  learning_rate = 0.01

  criterion = torch.nn.MSELoss() # mean-squared error for regression
  optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)

  # Train the model
  for epoch in range(num_epochs):
      outputs, h = model(trainX) # we don't use h there, but we can!
      optimizer.zero_grad()
      
      # obtain the loss function
      loss = criterion(outputs, trainY)
      loss.backward()
      
      optimizer.step()
      if epoch % 100 == 0:
          print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

input_size = 1
hidden_size = 4 
rnn = AirTrafficPredictor(input_size, hidden_size)
time_series_train(rnn)

### Testing

In [ ]:
def time_series_plot(train_predict):
  data_predict = train_predict.data.numpy()
  dataY_plot = dataY.data.numpy()

  # Denormalize
  data_predict = data_predict[0] *td_max + td_min
  dataY_plot = dataY_plot *td_max + td_min 
  #print(data_predict[:15])

  # Ploitting
  plt.figure(figsize=(12, 4))
  plt.axvline(x=train_size, c='r', linestyle='--')

  plt.plot(dataY_plot)
  plt.plot(data_predict)
  plt.suptitle('Time-Series Prediction')
  plt.show()

rnn.eval()
train_predict, h = rnn(dataX)
time_series_plot(train_predict)


[Time Series Prediction with LSTM Using PyTorchTime Series Prediction with LSTM Using PyTorch](https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/content/courses/deeplearning/notebooks/pytorch/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb#scrollTo=NabsV8O5BBd5https://colab.research.google.com/github/dlmacedo/starter-academic/blob/master/content/courses/deeplearning/notebooks/pytorch/Time_Series_Prediction_with_LSTM_Using_PyTorch.ipynb#scrollTo=NabsV8O5BBd5)



## Посимвольная генерация текстов

[Github RNN-walkthrough/main.ipynb](https://github.com/gabrielloye/RNN-walkthrough/blob/master/main.ipynb)

Одним из основных направлений использования рекуррентных сетей является работа с текстами:
- генерация (Language modeling)
и 
- перевод (Machine Translation)

Давайте посмотрим как решаются такого рода задачи.

Начнем с относительно простой - посимвольной генерации текста.

Постановка задачи:

предсказать следующий символ в последовательности.

- исходный текст:
'hey how are you'

- искаженный текст:

'hey how are yo'

- Верное предсказание:
'u'


Теоретически эту технику можно использовать для генерации подсказок при наборе текстов, исправления ошибок или восстановления частично утраченного текста.





<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-037.gif" width="400">

[ссылка](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

#### Подготовка данных

1. Зафиксировать словарь
2. Разбить данные
3. Кодирование символов 

In [ ]:
text = ['hey how are you','good i am fine','have a nice day']

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

print(char2int)

Вместо ASCII символа, каждой букве мы сопоставили номер.

### Выравнивание данных (Padding)

RNN допускают работу с данными переменной длины. Но что бы поместить предложения в batch надо их выровнять.


Обычно размер батча делают равным самому длинному предложению, а остальные просто дополняют пробелами до этого размера.

In [ ]:
maxlen = len(max(text, key=len))
print("The longest string has {} characters".format(maxlen))

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of the sentence matches
# the length of the longest sentence
for i in range(len(text)):
    while len(text[i])<maxlen:
        text[i] += ' '

print(text)



### Разбиение данных



В качестве входа будем использовать предложение без последнего символа:

**'hey how are yo'**

, а в качестве результата - предложение в котором он сгенерирован.

**'ey how are you'**


In [ ]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
    input_seq.append(text[i][:-1])
    
    # Remove firsts character for target sequence
    target_seq.append(text[i][1:])
    print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Как видим вравнивание служит здесь плохую службу.

### Кодирование

Теперь символы надо перевести в числа. Для этого мы уже построили словарь.

P.S. Запускать блок только один раз.

In [ ]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

print("Input",input_seq)
print("Target",input_seq)

#### One-hot-encoding(!)

Теперь из чисел надо сделать вектора. 


Почему бы не оставить числа?
В прошлом примере модель хорошо с ними работала.

В прошлом примере использовася MSE и на выходе было число.

Если бы мы определили отношение порядка над номерами букв, то что-то подобное можно было бы сделать.

Однако сейчас мы предсказываем класс буквы.
Поэтому на входе и на выходе должен быть вектор.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/softmax_1.png" width="250">

In [ ]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))
print(input_seq[0])

Каждый символ закодировали вектором.
Не слишьком экономно, зато удобно умножать на матрицу весов.

P.S. Запускать только один раз

In [ ]:
# Convert data to tensor
input_seq = torch.Tensor(input_seq)
target_seq = torch.Tensor(target_seq)

### Модель

In [ ]:
class NextCharacterGenerator(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super().__init__()

        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_size = hidden_dim, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)
        #Initializing hidden state for first input using method defined below
        hidden_0 = torch.zeros(1, batch_size, self.rnn.hidden_size) # 1 correspond to number of layers

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden_0)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        # Need Only if n_layers > 1
        out = out.contiguous().view(-1, self.rnn.hidden_size)
        out = self.fc(out)
        
        return out, hidden

### Обучение

In [ ]:
# Instantiate the model with hyperparameters
model = NextCharacterGenerator(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)

# Define hyperparameters
n_epochs = 100

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training Run
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    output, hidden = model(input_seq)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Тест

In [ ]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    
    out, hidden = model(character)
    #print(out.shape)
    #print(out)
    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

sample(model, 15, 'good')

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/softmax_2.png" width="700">

## $\color{brown}{\text{*Примеры применения}}$

Результаты которые удается получить при помощи моделей обученных на больших объемах данных.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-050.png" width="750">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-054.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-055.png" width="800">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-058.png" width="850">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-059.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-059-1.png" width="800">

[ссылка](https://github.com/torvalds/linux)

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-061.png" width="800">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-063.png" width="800">

### $\color{brown}{\text{*Explanation}}$

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-064.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-065.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-066.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-067.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-068.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-069.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-068.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-069.png" width="700">

[ссылка](https://arxiv.org/abs/1506.02078)

## $\color{brown}{\text{*Как пропускать градиент через RNN}}$

В случае с реккурентными сетями есть проблема с распространением градиента - если последовательность большая, то потребуется очень много времени

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-045.png" width="800">

Теоритетески можно было бы сразу пропустить все данные через сеть и затем вычислить градиент.

Но:
 - Большие последовательности не поместятся в памяти
 - Возникнут проблеммы исчезновения/взрыва градиента, так как цепочка будет очень длинной
 - Контекст затирается (по аналогии с ResNet)




Потому иногда делают пропускание градиента только через части последовательности, окнами. За счет этого получается меньшие затраты по памяти


<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-046.png" width="700">

Или можно пропускать только на сколько-то позиций от выхода - тоже можно, веса одни и те же используются, потому вполне возможно, что поправки от последних действий хватит, чтобы выправить нейросеть в целом

Также градиент в рекуррентных сетях быстро затухает. И при большой длине последовательности это становится критичным.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-047.png" width="700">

Скрытые состояния при этом сохраняются.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-083.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-086.png" width="1000">

Исчезающий/взрывающийся градиент (Vanishing/exploding gradient) &mdash; явления исчезающего и взрывающегося градиента часто встречаются в контексте RNN. Причина, по которой они происходят, заключается в том, что трудно уловить долгосрочные зависимости из-за мультипликативного градиента, который может экспоненциально уменьшаться/увеличиваться по отношению к числу слоев.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture12-023.png" width="700">

Градиентное отсечение (Gradient clipping) &mdash; метод, используемый для решения проблемы взрывающегося градиента, иногда возникающей при выполнении обратного распространения. Ограничивая максимальное значение градиента, это явление контролируется на практике.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gradient-clipping.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gradient-vanishing-exploding.png" width="700">

```
import torch.

threshold = 100 # Must be found empirically

for sequences, labels in train_loader:
        optimizer.zero_grad() 
        output = model(sequences)
        loss = criterion(output, labels)
        loss.backward() 
        # Add clippeng after backward
        torch.nn.utils.clip_grad_norm_(model.parameters(), threshold)
        # Before step
        optimizer.step()
```

[stackoverflow How to do gradient clipping in pytorch?](https://stackoverflow.com/questions/54716377/how-to-do-gradient-clipping-in-pytorch)

## LSTM



Ванильное RNN имело множество проблем, в том числе, в нем очень быстро затухала информация о предыдущих словах в предложении. Помимо этого, были проблемы с затухением/взрывом самого градиента. 

Эти проблемы были частично решены в LSTM

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-090.png" width="700">

Если в Vanilla RNN был только один путь, то в LSTM есть highway для сокращения информации  

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/lstm.png" width="800">

Была предложена концепция gate "врат", в которых решается, какая информация и как будет добавлена к главному потоку - $c$




* i = Input - преобразуем результат обработки текущего объекта и состояния h в сигнал от 0 до 1
* g = Gate - решаем, на основе результатов же обработки текущего объекта и состояния h, какую часть из преобразованной информации добавим/вычтем из потока c.  
* f = Forget - опять же, на основе результатов обработки текущего объекта  и состояния h решаем, какую часть информации из c можно забыть 
* o = Output - формируем на основе нового c и результатов обработки текущего объекта  и состояния h новое состояние h




1. Конкатенируем x и h_t-1
2. Умножаем на веса
3. Результат делим на 4 части (shape = hidden_size) к каждой применяем свою функцию активации
4. Далее комбинируем их с входами и выходами


<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture12-094.png" width="700">

[ссылка](https://medium.com/@saurabh.rathor092/simple-rnn-vs-gru-vs-lstm-difference-lies-in-more-flexible-control-5f33e07b1e57)

 i = Input  f = Forget  o = Output  g = Gate

 Операция "кружек с точкой" - это поэлементной умножение.

Таким образом мы получили магистраль для градиента аналогично ResNet.

Однако надо отметить что статья вышла в 1997г(!)


[Long short-term memory](https://www.bioinf.jku.at/publications/older/2604.pdf)

### LSTMCell

[pytorch LSTMCELL](https://pytorch.org/docs/stable/generated/torch.nn.LSTMCell.html)

Интерфейс отличается от RNNCell количеством входов и выходов


In [ ]:
import torch

lstm_cell = torch.nn.LSTMCell(input_size = 3, hidden_size = 4)
dummy_input = torch.randn(1,3) # batch, input_size
h_0 = torch.randn(1,4)
c_0 = torch.randn(1,4)
h, c = lstm_cell(dummy_input, (h_0,c_0)) # second arg is tuple
print("h",h.shape,h) # batch, hidden_size
print("c",c.shape,c) # batch, hidden_size



### LSTM in Pytorch

Отличие состоит в том что возвращается кроме h возвращается еще и c. Но можно использовать только output. 

In [ ]:
import torch
lstm = nn.LSTM(input_size = 3, hidden_size = 3)
dummy_input = torch.randn(2,1,3) # seq_len, batch, input_size
out, (h, c) = lstm(dummy_input) # h and c returned in tuple
print("out",out.shape,out) # seq_len, batch, hidden_size : h for each element
print("h",h.shape,h) # batch, hidden_size
print("c",c.shape,c) # batch, hidden_size

### Пример использования 

Что бы убедиться в работоспособности конструкции заменим RNN блок на LSTM в задаче предсказания временного ряда.

In [ ]:
# Define new LSTM based model
import torch
import torch.nn as nn

class LSTMAirTrafficPredictor(nn.Module):

    def __init__(self, input_size, hidden_size):
        # hidden_size == number of neurons 
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first = True)
        self.fc = nn.Linear(hidden_size, 1) # Predict only one value

    def forward(self, x):
        out, (h,c) = self.lstm(x) 
        y = self.fc(h)
        return y

lstm =  LSTMAirTrafficPredictor(input_size =1 , hidden_size =4 )
di = torch.randn((108,8,1))
out = lstm(di)
print(out.shape)


Train

In [ ]:
lstm.train()

num_epochs = 2000
learning_rate = 0.01

criterion = torch.nn.MSELoss() # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

 # Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX) 
    optimizer.zero_grad()
    #print(outputs.shape)
    loss = criterion(outputs, trainY.unsqueeze(0))
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
        print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))


In [ ]:
lstm.eval()
train_predict = lstm(dataX)
time_series_plot(train_predict)

### Выводы:
---
1. Использование LSTM слоев при прогнозировании нестационарных временных рядов - отличное решение, тк нейронка способна самостоятельно выденить динамики изменениях составляющих временного ряда. Правда, в более сложных случаях (трейдинге, например, или чтении и расшивровки волн мозга) приходится прибегать к feature ingineering
2. Сложость нейронной сети должна быть адекватна сложности подаваемых в нее данных. С ростом ансамбля и числа нейронов увеличивается заучивание тренировочной выборки и теряется способность к обощению
3. Предварительный анализ цикличности в данных (если она есть) помогает понять оптимальный размер ансамбля (тут видно, что цикл в среднем составляет 8 интервалов)
4. Также результат может зависеть от типа скалирования, который Вы применяете. Нужно знать принципы работы скаллеров и не стесняться экспериментировать с ними. См. статьи: [Data Preprocessing with Scikit-Learn: Standardization and Scaling](https://towardsdatascience.com/data-preprocessing-with-scikit-learn-standardization-and-scaling-cfb695280412) и [Gradient Descent, the Learning Rate, and the importance of Feature Scaling](https://towardsdatascience.com/gradient-descent-the-learning-rate-and-the-importance-of-feature-scaling-6c0b416596e1)
5. При всей выгодности приненении нейронных сетей, необходимо быть осторожным с автокорреляцией (см статью [Avoiding the pitfalls](https://towardsdatascience.com/how-not-to-use-machine-learning-for-time-series-forecasting-avoiding-the-pitfalls-19f9d7adf424))

## GRU (Gated reccurent unit)

LSTM выглядела очень громоздко. Потому решили искать более  компактную структуру. Так получили GRU

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/gan/gru1.png" width="700">

[ссылка](https://medium.com/@saurabh.rathor092/simple-rnn-vs-gru-vs-lstm-difference-lies-in-more-flexible-control-5f33e07b1e57)

In [ ]:
import torch
rnn = torch.nn.GRU(input_size = 4, hidden_size =3)
input = torch.randn(2, 1, 4) #seq_len, batch, input_size
h0 = torch.randn(1, 1, 3)
output, h = rnn(input, h0)

print("Out",output.shape,"\n",output) # seq_len = 2
print("h",h.shape,"\n",h) # last h


Иногда лучше работает GRU, иногда &mdash; LSTM

## Sequence-to-Sequence with RNNs

* Сейчас мы пытаемся решить задачу sequence to sequence
* Орабатывая входную последовательность, мы хотим обобщить всю информацию, которая в ней содержится в некий вектор С
* Далее мы передаем этот вектор во вторую RNN, которая является декодером


Мы используем вектор С для передачи информации между энкодером и декодером, также мы предполагаем, что вектор С обощает всю информацию, которая небходима декодеру для генерации выходных последовательности.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture13-011.png" width="800">

В качестве С и So может использоваться просто h_4 (последнее скрытое состояние)

### Машинный перевод
[pytorch NLP FROM SCRATCH: TRANSLATION WITH A SEQUENCE TO SEQUENCE NETWORK AND ATTENTION](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

дополнительно:
[pytorch LANGUAGE TRANSLATION WITH TORCHTEXT](https://pytorch.org/tutorials/beginner/torchtext_translation.html)

#### Загрузка и предобработка данных

In [ ]:
!wget https://download.pytorch.org/tutorial/data.zip
!unzip data.zip

In [ ]:
! sed -n 200,210p  data/eng-fra.txt

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Класс для хранения слов. Работает аналогично символам:
Создается словарь из всех слов и таким образом каждое слово получает свой индекс (номер).  

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

Вспомогательные методы для загрузки пар фраз из файла.

In [ ]:
import random

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(file_name):
  print("Reading lines...")

  # Read the file and split into lines
  lines = open(file_name, encoding='utf-8').\
      read().strip().split('\n')

  # Split every line into pairs and normalize
  pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

  # Make Lang instances
  input_lang = Lang('en')
  output_lang = Lang('fr')

  return input_lang, output_lang, pairs

MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(file_name):
    input_lang, output_lang, pairs = readLangs(file_name)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('data/eng-fra.txt')
for i in range(5):
  print(random.choice(pairs))

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Теперь в нашем распоряжении есть два словаря и набор пар строк.
Определим структуру модели.

In [ ]:
from torch import nn
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        print(input_size)
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden = None):
        embedded = self.embedding(input)
        print("Embedding: ",embedded.shape)
        output, hidden = self.gru(embedded.view(1, 1, -1), hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)

training_pair = pairs[0]

input_tensor = training_pair[0]
target_tensor = training_pair[1]

input_tensor = tensorFromSentence(input_lang, input_tensor)
target_tensor = tensorFromSentence(output_lang, target_tensor)

print("Input tensor",input_tensor.shape,input_tensor)

encoder_hidden = encoder.initHidden()
encoder_outputs, hidden = encoder(input_tensor[0])

print("Out",encoder_outputs.shape)
print("Hidden",hidden.shape)
#dummy_input = torch_randn()

Вместо one_hot - векторов, используются эмбеддинги размером 1x256 (hidden size)

[pytorch WORD EMBEDDINGS: ENCODING LEXICAL SEMANTICS](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, output_size, hidden_size ):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

decoder = DecoderRNN(output_lang.n_words, hidden_size) # hidden state must have the same sizes
decoder_input = torch.tensor([[SOS_token]], device=device)

# Use encoder last state as decoder init
decoder_hidden = encoder_hidden

decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden )
print("Output: ", decoder_output.shape)
#https://pytorch.org/docs/stable/generated/torch.topk.html
top_val, top_index = decoder_output.topk(1) # Returns the k largest elements of the given input tensor

generated_word = output_lang.index2word[top_index.item()]
print("Word: ", generated_word, "index ", top_index.item())


#### Обучение

В на вход модели будем подавать индексы слов для первой части пары из одного словаря, а на выходе ожидать индексы для соответствующей пары из другого. 

In [ ]:
target_length = target_tensor.size(0)
decoder_input = torch.tensor([[SOS_token]], device=device)
for di in range(target_length):
  decoder_output, decoder_hidden, = decoder( decoder_input, decoder_hidden)
  topv, topi = decoder_output.topk(1)
  decoder_input = topi.squeeze().detach()  # detach from history as input

  #loss += criterion(decoder_output, target_tensor[di])
  generated_word = output_lang.index2word[decoder_input.item()]
  print("Word: ", generated_word, "index ", decoder_input)

  if decoder_input.item() == EOS_token:
      break

## Attention

С подходом, разобранным ранее есть большая проблема

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/498_FA2019_lecture13-011.png" width="800">

Использование единственного вектора для предоставления информации может работать только для последовательностей малой длины (неразумно предполагать, что информация с длинной последовательностью можно сжать в один вектор) 

Чтобы понять, что надо сгенерировать слово estamos в начале, нашей нейросети надо сохранить информации о we во всех состояниях от $h_1$ до $h_4$
А что если нам надо перевести абзац текста? 
При этом длина вектора, в котором нам надо хранить информацию обо всей последовательности - постоянна. Понятно, что начиная с какого-то момента информацию всю мы сохранить в нем не сможем. Более того, при генерации, скажем, последнего слова, мы должны в векторе того же размера сохранить информацию о этом последнем слове и о том, что другие уже сгенерены - иначе модель может зациклиться или не сгенерировать часть слов.

Все эти проблемы реальны и возникают в обычной Seq2Seq модели.

Нам нужен некий механизм, который будет обходить "бутылочное горлышко" и не сжимать всю информацию в один вектор С. 

### Sequence-to-Sequence with RNNs and Attention mechanism

Давайте не будем делать состояние $c$ фиксированным. Пусть на каждом этапе, при генерации нового слова, наш декодер имеет возможность сформировать новое $c$, на основе всех скрытых состояний $h_i$ енкодера. 
Для простоты - пусть он подбирает веса, суммирующиеся в единицу, с которыми сложить эти представления, чтобы получить новое $c$.
Таким образом, в каждый момент времени наш декодер имеет доступ ко всему исходному предложению (представлению каждого слова) и может акцентировать внимание на конкретно нужных в данный момент словах. 

Говоря немного мудрено - теперь длина пути, который надо преодолеть информации от места ее появления до места, где она может пригодиться - 1, а раньше она была пропорциональна длине исходного предложения. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-017.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-020.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-021.png" width="700">

[ссылка](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf)

In [ ]:
import torch
from torch import nn

MAX_LENGTH = 17

class AttnDecoderRNN(nn.Module):
    def __init__(self, output_size, hidden_size,  max_length=MAX_LENGTH):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        # ****** Attention ***************************************************
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        # ********************************************************************

        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)

        # ****** Attention ***************************************************
        att_inputs = torch.cat((embedded[0], hidden[0]), 1) # S0 + Start token (Y0)
        print("att_inputs",att_inputs.shape,"256 + 256") # 256 + 256
        e = self.attn(att_inputs) 
        print("e ",e.shape) # MAX_INPUT == number of hidden states
        attn_weights = F.softmax( e, dim=1) 
        print("a (attn_weights) ",attn_weights.shape) # MAX_INPUT
        print("H (encoder_outputs)",encoder_outputs.unsqueeze(0).shape)
        # bmm is matrix product wark as elemet-wise multiplication + sum
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        print("C Attn_applied",attn_applied.shape)
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        print("C with YO",output.shape)

        # *********************************************************************
        

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# Fake hidden states from encoder
encoder_outputs = torch.randn(MAX_LENGTH, encoder.hidden_size, device=device)

att_rnn_decoder = AttnDecoderRNN(output_lang.n_words, hidden_size ) # output_size == word count in target dictionary
decoder_input = torch.tensor([[SOS_token]], device=device)
decoder_hidden = encoder_hidden

output, hidden, attn_weights = att_rnn_decoder(decoder_input, decoder_hidden, encoder_outputs)


Давайте посмотрим как фокусировка работает на примере перевода с английского на французский 

Как мы знаем, на каждом шаге генерируется набор весов, которые отвечают за фокусировку на том или ином месте входной последовательности. Как мы видим, английское предложение имеет иной порядок слов относительно французского. Например, в английском варианте словосочетание **European Economic Area**, в то время как во французском **zone économique européenne**. 

В английском прилагательные идут перед существительным, в то время как во французском языке наоборот. 

Таким образом, благодаря гибкости модели, мы можем обрабатывать и учитывать разный порядок слов в разных языках.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-025.png" width="700">

[ссылка](http://cs231n.stanford.edu/slides/2017/cs231n_2017_lecture10.pdf)

* Механизм внимания не обязательно должен принимать на вход последовательность. 

* Мы можем применять его, в том числе для генерации подписей для картинок. Входом в данном случае будет являться сетка признаков, которая была получена при применении сверточной сети к картинке. 

* Далее по этой сетке мы считаем веса внимания и делаем аналогично первому примеру.

Следующим шагом будет также заметить, что для достаточно длинного предложения наша модель может забыть и то, что она генерирует. Потому мы можем сделать два attention - один на представления исходного предложения, а второе - на представление того, что уже сгенерировано (что еще не сгенерировано - заменяем нулями)


## Проблема attention

Очевидно, у этого подхода есть свои минусы. 

Самый важный из них - ваша модель должна уметь предсказывать веса для каждого слова в исходном предложении. И обычные подходы принуждают нас фиксировать размер этого вектора весов. 

За счет этого у нас возникло ограничение, от которого мы избавлялись - модель не может принимать (и, если используем attention на то, что сгенерировано, генерировать) предложения больше определенного размера. С другой стороны, ничто не мешает поставить этот максимальный размер достаточно большим, чтобы для бОльшей части предложений это работало. 

Почему это не сильный проигрыш в сравненении с LSTM - так оно все равно было ограничено в длине предложения, просто неявно, что даже хуже - лучше иметь модель, которая на длине предложения 45 падает, а не генерирует лабуду. 




## Image Captioning with RNNs and Attention

Модели, основанные на внимании (attention) намного более продвинутые, нежели обычные нейросети. Они могут концентрироваться на отдельных частях изображения, что позволяет избежать зашумления данных.

Идея состоит в том, что свёрточная сеть теперь будет генерировать не один вектор, описывающий всё изображение, а набор векторов для нескольких участков исходного снимка. В дополнение к работе со словарём на каждом временном шаге модель также производит распределение по точкам на изображении, которые она обрабатывает в данный момент. Это позволяет ей научиться находить наиболее важные участки, на которых необходимо фокусироваться.

После обучения модели можно увидеть, что она как бы переносит своё внимание по изображению для каждого генерируемого слова.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-028.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-031.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-036.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-037.png" width="700">

[ссылка](http://cs231n.stanford.edu/slides/2017/)

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-038.png" width="700">


Также нейросети, основанные на внимании, повсеместно используются для ответов на визуальные вопросы (Visual Question Answering). Цель этой задачи — обучить модель отвечать на вопрос по изображению. Например, она должна уметь не только называть сами объекты на фотографии, но и считать их, распознавать цвета и оценивать расположение относительно друг друга. Мы уже рассказывали о подобных архитектурах в статье о том, как такие нейросети могут помочь незрячим людям и о нейро-символическом мышлении.

[ссылка на исследование](https://www.reg.ru/blog/nejroset-opisyvaet-mir-nezryachim-lyudyam/) 
и еще [одна ссылка](https://www.reg.ru/blog/uchim-nejroseti-rassuzhdat-o-tom-chto-oni-vidyat/)

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-039.png" width="700">

## Key, query, value 

Мы можем формализовать процедуру attention через введение концепций key, query 
и value. 

К примеру, у нас есть картинка. На этой картинки у нас есть области, которые можно описать одним словом - key. Например - фонарь/девушка/... 

Сами эти области - это value, которые введенным key соответствуют. 

Далее нам приходит query - например, running. Мы можем посчитать похожесть каждого из ключей, которые у нас есть на query. 

И далее выдать информацию только по value, похожим на наш query. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/1_EC04ZMiCnLBT3IG0tdU33g.jpg" width="800">


[Послезная статья про NLP — BERT & Transformer, откуда взяты картинки по теме](https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524)

По сути мы описали то, как будем делать при помощи нейронок питоновский словарь. С той разницей, что питоновский словарь может выдавать значения для только тех ключей, что в нем есть, а наш словарь выдает ответ для любого ключа-запроса, основываясь на его похожести на ключи словаря

### Keq, Query, Value на примере текста 

Разобьем наше предложение на токены (в данном случае - просто слова). Получим представление каждого токена

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/sentence_kqv.png" width="800">

Каждое слово будет являться одновременно и key, и value. 

Теперь можно подать любой query, например - вода, и в случае, если у нас эмбединги слов хорошие, что нам "подсветятся" слова, связанные с водой

## Attention Layer

Эту идею реализует attention layer. 





Что мы таким образом получили? Фактически, мы получили дифференцируемый аналог словаря в Python. Только этот словарь еще умеет делать неточный поиск

У нас есть некий изначальный набор X - слова в нашем предложении. 

Мы можем их преобразовать как-то, чтобы получить более удобные для дальнейшей цели K - предполагается, что они лучше помогают сравнивать между собой слова. 
В простейшем случае некое линейное преобразование $K = XW_K$

Далее у нас есть значения, V, которые тоже получаются из X путем какого-то преобразования, которое делает их более применимыми для работы модели (важные признаки и тд), например $V = XW_Q$



Далее к нам приходят запросы Q, которые находятся в том же пространстве, что и ключи. Фактически - наша нейросеть пытается вытащить из данных интересующую ее информацию. К примеру - ей нужно найти всех животных и все растения - тогда Q= {"животные", "растения"} 


Мы сравниваем эти запросы с ключами - считаем просто попарные косинусные расстояние между каждым ключом $K_i$ и запросом $Q_j$. Получаем похожесть каждого ключа на запрос, нормируем ее на корень из размерности представления нашего ключа. 

$E = \dfrac {QK^T} {\sqrt{D}} $

Получили матрицу похожестей $E$, где $E_{ij}$ - похожесть ключа $K_i$ на запрос $Q_j$

Далее мы применяем к похожестям $E$ softmax, беря его по каждомуу ключу отдельно (по каждому столбцу матрицы E). Теперь у нас похожести каждого запроса складываются в единицу. 

$A = softmax(E, dim=1)$

Получили для каждого запроса его "разложение" в виде ключей. 

Далее, чтобы получить значения, соответствующие каждому запросу, складываем значения с весами соответствующих ключей

$Y_j = \sum_iA_i V_i$

$Y = AV$

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-056.png" width="800">

[ссылка](http://cs231n.stanford.edu/slides/2017/)

### Softmax normalization 

Вы могли заметить, что в формуле для E мы делим на корень из размерности представления ключа. Зачем это? 



Давайте сгенерируем вектор из многомерного нормального распределения

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 

a = np.random.normal(0,100,size=(10000))
plt.hist(a)
plt.show()

И посмотрим, что будет с распределением значений этого вектора, если к нему применить softmax

In [ ]:
from scipy.special import softmax 

plt.plot(softmax(a))
plt.show()

Почти все значения 0 и одно (по случайным причинам), стало 1. 
Мы получили так называемое вырожденное распределение - у него вся плотность сосредоточена в одной точке. 

И получили на абсолютно случайных данных. 

Это приведет к затуханию градиента - мы будем распространять ошибку только для 1 значения из 10000. Учиться сеть будет плохо

Но этого можно избежать - давайте просто отнормируем наши данные 

In [ ]:
std = np.random.normal(0,100,size=(10000))

unit_std = std/100

plt.hist(std)
plt.show()
plt.hist(unit_std)
plt.show()

По сути в распределении ничего не поменялось - только масштаб. Но теперь softmax работает нормально 

In [ ]:
plt.hist(std)
plt.show()

Остается только понять, а как нормировать наши данные в нашем слое? Считать налету, наверное, не лучшая идея. 

Наш слой делает сколярное произведение между двумя векторами, предположим, нормально распределенными

In [ ]:
c = []
for i in range(100):
	    a = np.random.normal(0,1,size=(100))
	    b = np.random.normal(0,1,size=(100))
	    c.append(np.dot(a,b))

In [ ]:
print(np.mean(c))

print(np.std(c))

Видим, что среднее не сильно отличается от 0 (можно показать, что в среднем оно равно 0 для произведения нормально распределенных величин). 

А вот стандартное отклонение растет. И, можно предположить, что растет оно как корень из размерности вектора. Так и есть. Потому и появляется именно такой нормировочный множитель в attention

### Keq, Query, Value на примере текста. Второй заход

Хорошо, у нас есть предложение, мы его разбили на токены, допустим, мы хорошо умеем кодировать эти токены. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/sentence_kqv.png" width="800">

Но каждый токен закодирован на основе самого себя, и только, например, слово bank встречается в тексте два раза, и будет закодировано одинаково, хотя несет разные значения (берег и, собственно, банк). 

Как нейросети их отличать? Ну, если мы попытаемся на основе только одного токена что-то сделать - никак. 

А как улучшить кодирование каждого токена, чтобы два "банка" отличались? 

Сделаем следующее - каждый токен подадим в нейросеть **и в качестве query**. 

И будем аггрегировать полученную информацию, получая токен, который будет содержать более конкретную информацию

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/sentence_kqv2.png" width="800">

Например, bank, который банк, будет содержать информацию о том, что он не просто bank, а "Bank of America", который нужно найти (find).

Такой подход - когда key, query и value формируются на основе одного и того же входа нейросети называется self-attention




## Self-Attention Layer

Сделаем это все в виде слоя

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-063.png" width="800">

[ссылка](http://cs231n.stanford.edu/slides/2017/)

Что изменилось? 

Теперь X участвует и в производстве K, и в производстве V, и в производстве Q. Потому и self-attention - предложение состоящее из слов X_i, ищет само себя в нашем дифференцируемом словаре. Почему это не приводит к тому, что мы просто получим вектор V?
Потому что значения X_i модифицируются и K != Q.

Еще один пример проблемы с текстом, который обычные методы решали плохо

Многие из нас знают Первый Закон Робототехники

**Second Law of Robotics**


A robot must obey the orders given it by human beings except where such orders would conflict with the First Law.

Представим, что машина хочет понять, что здесь написано. И ей нужно понять, что имеется ввиду под it. 






Первым делом она формирует на основе каждого слова key и value. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/self-attention-example-folders-3.png" width="800">

Затем она смотрит, какие ключи похожи на Query, полученный из it

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/self-attention-example-folders-scores-3.png" width="800">

Чтобы получить, что же на самом деле значит it, машина считает взвешенную сумму values, где  веса определяются похожестью query на value

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/gpt2-value-vector-sum.png" width="800">

[Ссылка на оригтнальную статью про GPT-2 с более подробными объяснениями как оно работает.](https://jalammar.github.io/illustrated-gpt2/)

В итоге получается, что it представляет собой "a robot" (ну и само себя). 

### Positional encoding

Единственный возможный минус - наша нейросеть не учитывает порядка слов в предложении при составлении embedding. Это может нам мешать. Например, если в предложении два it, то они часто относятся к разным словам. Потому хотим ууметь учитывать информацию о позиции. Для этого к X при составлении Q добавляется информация о позиции. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-072.png" width="700">

[ссылка](http://cs231n.stanford.edu/slides/2017/)

Делается это хитрым образом - мы берем и добавляем к каждому значению исходного вектора токенов некую комбинацию sin и cos с разными параметрами. 

Вектор PE, который мы будем добавлять к X будет определяться по следующей формуле

$$PE_{pos, 2i} = \sin {\dfrac {pos} {10000^{2i/d}}}$$

$$PE_{pos, 2i+1} = \cos {\dfrac {pos} {10000^{2i/d}}}$$

$pos$ - это позиция токена

$i$ - iя размерность токена

In [ ]:
import torch 
import math
import torch.nn as nn
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].detach() 
        return x

In [ ]:
pe = PositionalEncoding(20)
y = pe(torch.zeros(1, 100, 20)) # последовательность размера 100, каждый токен последовательности имеет размерность 20 

In [ ]:

plt.figure(figsize=(15, 5))
plt.plot(np.arange(100), y[0, :, 0:4].data.numpy())
plt.legend(["dim %d"%p for p in [1,2,3,4]])
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(np.arange(100), y[0, :, 4:8].data.numpy())
plt.legend(["dim %d"%p for p in [4,5,6,7]])
plt.show()

Это помогает трснформеру достаточно уникальным образом определять каждую позицию и понимать относительное расстояние между разными токенами

### Masked Self-Attention Layer


Если у нас стоит проблема, что мы не должны видеть часть слов в предложении - например, при генерации текста (по текущим словам предсказать следующее). Например, хотим сгенерировать фразу "robot must obey orders" на основе только первого слова. 

Если мы скормим в трансформер во время обучения сразу все предложение, которое хотим генерировать, то у нас в первых словах будет "протекать" информация о предыдущих.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/queries-keys-attention-mask.png" width="800">

В этом случае мы можем просто на соответствующих местах матрицы E поставить минус бесконечности - тогда в ембедингах слов, которые не должны знать о каких-то словах, информации об этих словах не будет.

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/transformer-attention-mask.png" width="800">

В результате мы сможем, после softmax "лишния" информация не будет использоваться при генерации ответа на query

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/transformer-attention-masked-scores-softmax.png" width="800">

Благодаря этому трюку, у нас получается обучать transfomer по-прежнему как простую single-pass нейросеть, а не "скатываться" в RNN, где у нас возникнут проблемы с градиентами и временем работы 

## Multihead Self-Attention Layer

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/l08_7-1.png" width="700">

Давайте посмотрим на то, к каким словам предложения gave может иметь отношение. В общим случае глагол может иметь связку со многоми частями предложения. Как, например, к подлежащему, так и к причастиям.

В идеале, нам бы хотелось обратить внимние функции (attention) на все эти взаимосвязи. Для этого нам просто надо поставить несколько attention слоев парралельно. В идеале каждый будет учить что-нибудь свое - аналогия с конволюциями. 

* Чтобы осуществить задуманное, вместо одного набора query, будем использовать несколько независимых наборов. 

* Причем каждый набор будет считаться уникальной матрицей. 

* Аналогично сделаем для keys и values. Количество таких наборов внутри keys, queries, values должно быть **одинаковым**. 

* Обозначим это число как h - head, далее производим аналогичные манипуляции, при этом введем в параллель h таких функций attention
* На последнем шаге мы их соединяем (конкатинируем)

* При этом, можно заметить, что таком подходе на каждом шаге размерность токена будет увеличиваться (если, например, в качестве и key, и value, и query мы подаем одно и тоже предсталение токена). Если хотим сохранять управлять размерностью токена, то придется получать по меньшей мере value путем домножения на матрицу, размерность которой по второй оси меньше - **выполнять проекцию наших токенов в пространство меньшей размерности**

* В частности, можно подобрать размерность этого пространства таким образом, чтобы при конкатенации размерность полученного токена равнялась исходной. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/Multihead.png" width="900">

## Attention is all you need

Оказывается, этот подход работает сам по себе. Не нужно добавлять никаких реккурентных слоев - просто делаем много правильно соединенных attention layers. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-085.png" width="800">

### Базовый блок 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img/498_FA2019_lecture13-092.png" width="700">

[ссылка](http://cs231n.stanford.edu/slides/2017/)

Блок устроен следующим образом: входные вектора мы запускаем в блок self-attention и используем residual connection как обходной путь вокруг self-attention, затем мы их складываем. Так как обычно используют multi head self-attention, тут нам и пригождается возможность подобрать параметры так, чтобы размерности входных и выходных токенов не отличались. 

После - применяем слой нормализации, затем - слой feed forward (MLP), плюс обходной путь вокруг feed forward. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/mlp_transformer.png" width="900">

Складываем выход MLP-части с residual connection и выполняем нормализацию.

Нормализация везде нужна для того, чтобы нейросеть нормально обучалась. 

Заводим ли разный MLP на каждый токен (по позиции) - конечно нет. К каждому токену применяются MLP с одними и теми же весами. 


В реальных моделях эти слои стекируют друг с другом и получается большая трансформер-модель. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/mlp_transformer2.png" width="600">

Это пример Encoder-модели. Такова рассматриваемая далее BERT. В такой модели на выходе мы получаем некое хорошее представление исходного предложения. 

Но что если мы хотим сделать задачу перевода с одного языка на другой? Нужен еще Decoder



## Decoder 



## Первая стадия

На первой стадии то, как работает Decoder не так сильно отличается от работы Encoder. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/1_R6_u8qGXKUyyGaHb9-mMDw.jpg" width="800">

Тем не менее, есть один важный нюанс -  мы используем Masked Attention - наша нейросеть не должна видеть слов, которые еще не сгенерировала. 

На основе первого этапа у нас формируются только query - наша нейросеть на основе того, что она уже сгенерировала решает, какая информация ей нужна дальше 

## Вторая стадия

Теперь мы используем для K, V информацию, полученную из энкодера и получаем предсказание. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/1_HKyS_RuocFun1LUubSf-jQ.jpg" width="800">

[ссылка](https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524)

# Примеры применений Transformer

## Перевод текста 

Непосредственное применение разобранной архитектуры Encoder-Decodee

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/transformer-encoder-decoder.png" width="800">

[ссылка](https://jalammar.github.io/illustrated-bert/)

## BERT 

В случае BERT используется только Encoder часть

Это нейросеть, предобученная на огромном корпусе английского текста. 

Перед ней ставили следующие задачи:

1. На вход дается предложение - в нем выбрано 15% токенов, из которых:
    1. 80% замаскированы
    2. 10% заменены случайным
    3. 10% оставлены без изменений




2. На вход даются два предложения - сказать, идет ли второе непосредственно за первым в тексте, или нет (просто случайное предложение из корпуса)


В результате на вход подается все в таком виде:

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/1_D0_sVWpmOSaGCvm6gk9aHA.jpg" width="800">


[ссылка](https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524)

$CLS$ токен нужен для того, чтобы нейросети было, куда класть информацию обо всем предложении в целом. 

$SEP$ нужен просто для того, чтобы разделять два предложения (если мы подаем их два) и для того, чтобы отмечать окончание. Использование $SEP$ с двумя целями позволяет подавать в уже обученную нейросеть только одно предложение. 


Выход нейросети во время обучения выглядит таким образом: 


### Masked Language Model

Для первой задачи используем Encoder-Decoder 

То есть у нас есть **Encoder**, который получает богатые представления и добавленный только на время обучения **Decoder** (не attention, просто MLP).

Именно Decoder отвечает за то, чтобы предсказывать пропущенные/замененные токены. Ошибка считается только по тем 15% токенов, для которых могло произойти изменения, а не по всему предложению.

Как гарантируется, что модель не заменяет имевшиеся в предложение слова на другие? 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/1_yTk92fAvqPDlabJJqkM4rw.png" width="800">


### Next Sentence Prediction 

Для второй задачи - Classifier. 

При этом на вход ему подается только сам CLS токен 


<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/1_UvFUs9afyoIGKj9F5qTIxw.png" width="800">


Обе задачи тренируем одновременно. 


### Transfer learning с BERT 

Нейросеть, обученную на таких задачах, можно было далее использовать при помощи transfer learning для решения целого спектра задач.

Обученную таким образом модель (оставляем только encoder), можно использовать для огромного числа других задач 


<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/1_71lj0YoM5fkZB229XKAFrA.jpg" width="800">

### Zero-shot learning 

Более того, есть BERT, тренированная на большом числе разных языков.

Это позволяет, например, сравнивать предложения из разных языков, хотя мы этому даже не учились. Поиграть можно [здесь](https://colab.research.google.com/github/deepmipt/dp_tutorials/blob/master/Tutorial_2_DeepPavlov_BERT_transfer_learning.ipynb#scrollTo=S1iqGcxUINyU). Веса будут грузиться ДОЛГО

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/BERT_multilingual.png" width="800">

### Специализированные аналоги BERT

Более того, можно тренировать BERT под строго определенные задачи - например, анализ текстов научных статей 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/0_svTduALJlF5JOD7x.png" width="800">

## GPT (Generative Pretrained Transformer )

В случае GPT используется только Decoder часть. Но теперь во всех частях используются masked attention 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/gpt2-self-attention-qkv-1-2.png" width="800">

Это нейросеть обширно используется для, например, задачи генерации текста,

Работает она следующим образом - даем ей в качестве исходного входа какую-то затравку. Можно просто SOS-токен. Она дает какой-то выход. Подаем этот выход нейросети как вход. 
Делаем так до тех пор, пока не надоест/не достигнем максимальной длины предложения/не встретим символ окончания генерации

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/gpt-2-autoregression-2.gif" width="800">

Как ее обучали - на самом деле, тоже unsupervised. Но теперь перед моделью ставится задача предсказывать по предыдущим словам в предложении текущее.

Понятно, что такую модель можно сразу же идти и использовать для генерации. 

Однако затем ее можно применять для многих других задач (опять же, путем transfer learning):


1. Классификации - подаем сразу все предложение, полученное представление используем для предсказания

2. Entailment (Определение логического следования) - даем изначальные данные, гипотезу, надо оценить, следует ли гипотеза из данных

3. Similarity - можем оценивать похожесть предложений. Так как это мы определяем порядок предложений, то чтобы не было произволо скормим будем использовать результат пропускания через нейросеть обоих возможных порядков. 

4. Выбор варианта ответа на вопросы - может обучить нейросеть отвечать на вопросы с множественным выбором. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/Screen_Shot_2020-05-27_at_12.41.44_PM.png" width="800">

Также можно научить нейросеть отвечать на вопросы и т.д. 

Строго говоря, мы можем даже текст переводить с помощью GPT. 

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/decoder-only-transformer-translation.png" width="800">

Просто подаем предложение с токеном в конце, определяющим, на какой язык переводим

По аналогичной схеме можем научить нашу сеть делать summary текста

<img src ="http://edunet.kea.su/repo/src/L08_RNN/img_license/decoder-only-summarization.png" width="800">

[А тут можно ознакомиться с полной статьей How NLP Cracked Transfer Learning](https://miro.medium.com/max/933/1*71lj0YoM5fkZB229XKAFrA.jpeg) , из которой взяты изображеня для блоков Masked Language Model, Next Sentence Prediction, Transfer learning с BERT, Zero-shot learning, Специализированные аналоги BERT и GPT

[Здесь](https://6b.eleuther.ai/) можно поиграть с open-source GPT-J

А [здесь](https://colab.research.google.com/github/kingoflolz/mesh-transformer-jax/blob/master/colab_demo.ipynb#scrollTo=e-NKauYvgTNG) - поиграть, но уже в Collab

# Хорошие источники 

[Про трансформеры](https://www.notion.so/Transformers-969f4b27c48147778c1e2dbda0c83ce0)

[Аннотированный трансформер](http://nlp.seas.harvard.edu/2018/04/03/attention.html)

[Код множества моделей с красивыами комментариями](https://nn.labml.ai/)

[BERT](https://medium.com/@samia.khalid/bert-explained-a-complete-guide-with-theory-and-tutorial-3ac9ebc8fa7c)

[Зоопарк BERT](https://ai.plainenglish.io/so-how-is-bert-different-ad43a42cab48)

[От NLP до Transformer](https://jonathan-hui.medium.com/nlp-bert-transformer-7f0ac397f524)

[Illustrated transformer](https://jalammar.github.io/illustrated-transformer/)

[Illustrated GPT-2](https://jalammar.github.io/illustrated-gpt2/)

[Open-source реализация GPT-3](https://arankomatsuzaki.wordpress.com/2021/06/04/gpt-j/)

[Transformer для русского языка](https://github.com/vlarine/transformers-ru)